In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import neighbors
import csv
import json
import os
import pandas as pd
import multiprocessing


In [2]:
raw_train_data = pd.read_csv('train_dataset.csv')

In [3]:
raw_train_data.head()

,用户编码,用户实名制是否通过核实,用户年龄,是否大学生客户,是否黑名单客户,是否4G不健康客户,用户网龄（月）,用户最近一次缴费距今时长（月）,缴费用户最近一次缴费金额（元）,用户近6个月平均消费值（元）,...,当月是否景点游览,当月是否体育场馆消费,当月网购类应用使用次数,当月物流快递类应用使用次数,当月金融理财类应用使用总次数,当月视频播放类应用使用次数,当月飞机类应用使用次数,当月火车类应用使用次数,当月旅游资讯类应用使用次数,信用分
0,a4651f98c82948b186bdcdc8108381b4,1,44,0,0,0,186,1,99.80,163.86,...,1,1,713,0,2740,7145,0,0,30,664
1,aeb10247db4e4d67b2550bbc42ff9827,1,18,0,0,1,5,1,29.94,153.28,...,0,0,414,0,2731,44862,0,0,0,530
2,5af23a1e0e77410abb25e9a7eee510aa,1,47,0,0,0,145,1,49.90,109.64,...,0,0,3391,0,0,4804,0,0,1,643
3,43c64379d3c24a15b8478851b22049e4,1,55,0,0,0,234,1,99.80,92.97,...,1,1,500,0,1931,3141,0,0,5,649
4,f1687f3b8a6f4910bd0b13eb634056e2,1,40,0,0,0,76,1,49.90,95.47,...,1,0,522,0,64,59,0,0,0,648


In [4]:
print(raw_train_data.shape)

(50000, 30)


In [5]:
#训练集数据和标签 pandas
raw_x = raw_train_data.iloc[:,1:29].astype(float)
raw_y = raw_train_data.iloc[:,29].astype(int)

In [6]:
raw_x.columns

Index(['用户实名制是否通过核实', '用户年龄', '是否大学生客户', '是否黑名单客户', '是否4G不健康客户', '用户网龄（月）',
       '用户最近一次缴费距今时长（月）', '缴费用户最近一次缴费金额（元）', '用户近6个月平均消费值（元）', '用户账单当月总费用（元）',
       '用户当月账户余额（元）', '缴费用户当前是否欠费缴费', '用户话费敏感度', '当月通话交往圈人数', '是否经常逛商场的人',
       '近三个月月均商场出现次数', '当月是否逛过福州仓山万达', '当月是否到过福州山姆会员店', '当月是否看电影', '当月是否景点游览',
       '当月是否体育场馆消费', '当月网购类应用使用次数', '当月物流快递类应用使用次数', '当月金融理财类应用使用总次数',
       '当月视频播放类应用使用次数', '当月飞机类应用使用次数', '当月火车类应用使用次数', '当月旅游资讯类应用使用次数'],
      dtype='object')

In [7]:
# 新增！差值特征
diff_feature_list = ['1', '2', '3','4','5','6','7','8','9','10','11','12','top_up_amount_offline'
                    ]

def add_diff_feature(data):
    data['1'] = data['缴费用户最近一次缴费金额（元）'] - data['用户近6个月平均消费值（元）']
    data['2'] = data['缴费用户最近一次缴费金额（元）'] - data['用户账单当月总费用（元）']
    data['3'] = data['缴费用户最近一次缴费金额（元）'] - data['用户当月账户余额（元）']
    data['4'] = data['用户近6个月平均消费值（元）'] - data['用户账单当月总费用（元）']
    data['5'] = data['用户近6个月平均消费值（元）'] - data['用户当月账户余额（元）']
    data['6'] = data['用户当月账户余额（元）'] - data['用户账单当月总费用（元）']
    

    data['7'] = data['缴费用户最近一次缴费金额（元）'] / (data['用户近6个月平均消费值（元）'] + 1) * 100
    data['8'] = data['缴费用户最近一次缴费金额（元）'] / (data['用户账单当月总费用（元）'] + 1) * 100
    data['9'] = data['缴费用户最近一次缴费金额（元）'] / (data['用户当月账户余额（元）'] + 1) * 100
    data['10'] = data['用户近6个月平均消费值（元）'] / (data['用户账单当月总费用（元）'] + 1) * 100
    data['11'] = data['用户近6个月平均消费值（元）'] / (data['用户当月账户余额（元）'] + 1) * 100
    data['12'] = data['用户当月账户余额（元）'] / (data['用户账单当月总费用（元）'] + 1) * 100
    

#age and net_age_in_month ---> 入网时的年龄 --- useless
#top up amount, 充值金额是整数，和小数，应该对应不同的充值途径？
    data['top_up_amount_offline'] = 0
    data['top_up_amount_offline'][(data['缴费用户最近一次缴费金额（元）'] % 10 == 0)&\
                               data['缴费用户最近一次缴费金额（元）'] != 0] = 1


In [8]:
# 训练集 新增 差值特征：
print(raw_x.shape)
add_diff_feature(raw_x)
print(raw_x.shape)
print(raw_x.columns)
raw_x.head()

(50000, 28)
(50000, 41)
Index(['用户实名制是否通过核实', '用户年龄', '是否大学生客户', '是否黑名单客户', '是否4G不健康客户', '用户网龄（月）',
       '用户最近一次缴费距今时长（月）', '缴费用户最近一次缴费金额（元）', '用户近6个月平均消费值（元）', '用户账单当月总费用（元）',
       '用户当月账户余额（元）', '缴费用户当前是否欠费缴费', '用户话费敏感度', '当月通话交往圈人数', '是否经常逛商场的人',
       '近三个月月均商场出现次数', '当月是否逛过福州仓山万达', '当月是否到过福州山姆会员店', '当月是否看电影', '当月是否景点游览',
       '当月是否体育场馆消费', '当月网购类应用使用次数', '当月物流快递类应用使用次数', '当月金融理财类应用使用总次数',
       '当月视频播放类应用使用次数', '当月飞机类应用使用次数', '当月火车类应用使用次数', '当月旅游资讯类应用使用次数', '1',
       '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       'top_up_amount_offline'],
      dtype='object')


D:\Python\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,用户实名制是否通过核实,用户年龄,是否大学生客户,是否黑名单客户,是否4G不健康客户,用户网龄（月）,用户最近一次缴费距今时长（月）,缴费用户最近一次缴费金额（元）,用户近6个月平均消费值（元）,用户账单当月总费用（元）,...,4,5,6,7,8,9,10,11,12,top_up_amount_offline
0,1.0,44.0,0.0,0.0,0.0,186.0,1.0,99.80,163.86,159.20,...,4.66,-16.14,20.80,60.536213,62.297129,55.138122,102.284644,90.530387,112.359551,0
1,1.0,18.0,0.0,0.0,1.0,5.0,1.0,29.94,153.28,145.10,...,8.18,43.28,-35.10,19.406274,20.492813,26.972973,104.914442,138.090090,75.290897,0
2,1.0,47.0,0.0,0.0,0.0,145.0,1.0,49.90,109.64,120.20,...,-10.56,39.64,-50.20,45.101229,41.171617,70.281690,90.462046,154.422535,57.755776,0
3,1.0,55.0,0.0,0.0,0.0,234.0,1.0,99.80,92.97,167.42,...,-74.45,2.97,-77.42,106.204108,59.256620,109.670330,55.201283,102.164835,53.437834,0
4,1.0,40.0,0.0,0.0,0.0,76.0,1.0,49.90,95.47,101.00,...,-5.53,15.47,-21.00,51.725925,48.921569,61.604938,93.598039,117.864198,78.431373,0


In [9]:
#训练集数据和标签 从pandas到numpy array
x = np.array(raw_x)
y = np.array(raw_y)

In [10]:
#划分出训练集和验证集
from sklearn.model_selection import train_test_split
x_train, x_validation, y_train, y_validation = train_test_split(x, y, test_size=0.10, random_state=6)

In [11]:
#读入测试数据
raw_test_data = pd.read_csv('test_dataset.csv')
raw_test_data.head()

,用户编码,用户实名制是否通过核实,用户年龄,是否大学生客户,是否黑名单客户,是否4G不健康客户,用户网龄（月）,用户最近一次缴费距今时长（月）,缴费用户最近一次缴费金额（元）,用户近6个月平均消费值（元）,...,当月是否看电影,当月是否景点游览,当月是否体育场馆消费,当月网购类应用使用次数,当月物流快递类应用使用次数,当月金融理财类应用使用总次数,当月视频播放类应用使用次数,当月飞机类应用使用次数,当月火车类应用使用次数,当月旅游资讯类应用使用次数
0,7171737d49b143d1b38883a39e4a5730,1,30,0,0,0,22,1,99.8,69.53,...,0,1,0,281,0,74,807,0,0,2
1,3af0a449d5424488912e8fb2bf4b9faa,1,70,0,0,0,84,0,0.0,12.89,...,0,0,0,0,0,0,0,0,0,0
2,eb2cf02e0d5c4d1294dd73e776dbb441,1,35,0,0,0,237,0,0.0,161.71,...,1,1,1,186,0,943,261,0,0,0
3,9c0f780ecb254670a11aa9e3f10777c5,1,44,0,0,0,161,0,0.0,229.60,...,1,1,1,244,0,72,591,0,0,3
4,d794eed46c1e44f785a575f18b3023a5,1,44,0,0,0,153,1,99.8,105.42,...,0,1,1,132,0,0,88,0,0,0


In [12]:
#划分测试数据和id pandas
raw_test = raw_test_data.iloc[:,1:29].astype(float)
raw_id = raw_test_data.iloc[:,0]

In [13]:
add_diff_feature(raw_test)
raw_test.head()

D:\Python\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,用户实名制是否通过核实,用户年龄,是否大学生客户,是否黑名单客户,是否4G不健康客户,用户网龄（月）,用户最近一次缴费距今时长（月）,缴费用户最近一次缴费金额（元）,用户近6个月平均消费值（元）,用户账单当月总费用（元）,...,4,5,6,7,8,9,10,11,12,top_up_amount_offline
0,1.0,30.0,0.0,0.0,0.0,22.0,1.0,99.8,69.53,50.38,...,19.15,-120.47,139.62,141.500071,194.239004,52.251309,135.325029,36.403141,369.793694,0
1,1.0,70.0,0.0,0.0,0.0,84.0,0.0,0.0,12.89,34.06,...,-21.17,-7.11,-14.06,0.000000,0.000000,0.000000,36.765545,61.380952,57.045066,0
2,1.0,35.0,0.0,0.0,0.0,237.0,0.0,0.0,161.71,171.09,...,-9.38,41.71,-51.09,0.000000,0.000000,0.000000,93.968272,133.644628,69.730955,0
3,1.0,44.0,0.0,0.0,0.0,161.0,0.0,0.0,229.60,223.00,...,6.60,9.60,-3.00,0.000000,0.000000,0.000000,102.500000,103.891403,98.214286,0
4,1.0,44.0,0.0,0.0,0.0,153.0,1.0,99.8,105.42,103.60,...,1.82,55.42,-53.60,93.779365,95.411090,195.686275,100.783939,206.705882,47.801147,0


In [14]:
x_test = np.array(raw_test)
id_test = np.array(raw_id)

In [15]:
#输出到文件
data_dir = 'preprocessed_add_diff'
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

np.save(os.path.join(data_dir,'x_train.npy'),x_train)
np.save(os.path.join(data_dir,'x_validation.npy'),x_validation)
np.save(os.path.join(data_dir,'y_train.npy'),y_train)
np.save(os.path.join(data_dir,'y_validation.npy'),y_validation)

np.save(os.path.join(data_dir,'x_test.npy'),x_test)
np.save(os.path.join(data_dir,'id_test.npy'),id_test)

In [16]:
#从文件load到numpy array
data_dir = 'preprocessed_add_diff'
#读入训练数据
x_train = np.load(os.path.join(data_dir,'x_train.npy'))
x_validation = np.load(os.path.join(data_dir,'x_validation.npy'))
y_train = np.load(os.path.join(data_dir,'y_train.npy'))
y_validation = np.load(os.path.join(data_dir,'y_validation.npy'))
#读入测试数据
x_test = np.load(os.path.join(data_dir,'x_test.npy'))
id_test = np.load(os.path.join(data_dir,'id_test.npy'))

print(x_train[0])
print(y_train[0])
print(x_test[0])
print(id_test[0])


[ 1.00000000e+00  3.20000000e+01  0.00000000e+00  0.00000000e+00
  0.00000000e+00  1.86000000e+02  1.00000000e+00  2.99400000e+01
  1.61400000e+02  1.61100000e+02  3.00000000e+01  0.00000000e+00
  3.00000000e+00  1.01000000e+02  0.00000000e+00  4.00000000e+00
  0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
  0.00000000e+00  1.73000000e+02  0.00000000e+00  6.85000000e+02
  2.58300000e+03  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -1.31460000e+02 -1.31160000e+02 -6.00000000e-02  3.00000000e-01
  1.31400000e+02 -1.31100000e+02  1.84359606e+01  1.84700802e+01
  9.65806452e+01  9.95681678e+01  5.20645161e+02  1.85070944e+01
  0.00000000e+00]
677
[   1.           30.            0.            0.            0.
   22.            1.           99.8          69.53         50.38
  190.            0.            4.           19.            0.
    2.            0.            0.            0.            1.
    0.          281.            0.           74.          807.
    0.     